In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass(prompt='Enter OpenAI API key: ')

In [13]:
from langchain import OpenAI
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key = os.environ["OPENAI_API_KEY"],
    temperature = 0,
    model_name = "gpt-4"
)

In [14]:

from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [15]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent = "zero-shot-react-description", #zero shot: no memory, uses ReAct to decide which tool to use
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 3
)

In [16]:
zero_shot_agent("What is (3.5*114.2)^-2.4?")



> Entering new AgentExecutor chain...
I need to calculate the expression (3.5*114.2)^-2.4. I can use the calculator for this.
Action: Calculator
Action Input: (3.5*114.2)^-2.4
Observation: Answer: 5.69951687933452e-07
Thought:I now know the final answer.
Final Answer: 5.69951687933452e-07

> Finished chain.


{'input': 'What is (3.5*114.2)^-2.4?', 'output': '5.69951687933452e-07'}

In [22]:
import pandas as pd
from sqlalchemy import create_engine

data = pd.read_csv("spotify-2023.csv", encoding='latin-1')
engine = create_engine('sqlite:///:memory:')

data.to_sql('Spotify_Top_Songs_2023', engine, if_exists='replace', index = False)

953

In [27]:
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain.from_llm(llm = llm, db = db, verbose = True)

In [28]:
from langchain.agents import Tool

sql_tool = Tool(
    name = "Spotify Top Songs 2023 DB",
    func = sql_chain.run,
    description = "Useful for when you need to answer questions about the Spotify top songs for 2023"
)

In [29]:
tools = load_tools(
    ["llm-math"],
    llm = llm
)

tools.append(sql_tool)

In [30]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key = "chat_history")

In [31]:
conversational_agent = initialize_agent(
    agent='conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)

In [32]:
result = conversational_agent(
    "Who was the artist with the top song on Spotify in April 2023?"
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Spotify Top Songs 2023 DB
Action Input: Top song artist, April 2023

> Entering new SQLDatabaseChain chain...
Top song artist, April 2023
SQLQuery:SELECT "artist(s)_name" FROM "Spotify_Top_Songs_2023" WHERE released_month = 4 AND released_year = 2023 ORDER BY streams DESC LIMIT 1
SQLResult: [('The Weeknd, Future',)]
Answer:The top song artist in April 2023 was The Weeknd, Future.
> Finished chain.

Observation: The top song artist in April 2023 was The Weeknd, Future.
Thought:Do I need to use a tool? No
AI: The artist with the top song on Spotify in April 2023 was The Weeknd, Future.

> Finished chain.


In [33]:
result = conversational_agent(
    "Who was the artist with the second top song in the same month?"
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Spotify Top Songs 2023 DB
Action Input: Who was the artist with the second top song in April 2023?

> Entering new SQLDatabaseChain chain...
Who was the artist with the second top song in April 2023?
SQLQuery:SELECT "artist(s)_name" FROM "Spotify_Top_Songs_2023" WHERE released_month = 4 ORDER BY streams DESC LIMIT 1 OFFSET 1
SQLResult: [('IU, Agust D',)]
Answer:The artist with the second top song in April 2023 was IU, Agust D.
> Finished chain.

Observation: The artist with the second top song in April 2023 was IU, Agust D.
Thought:Do I need to use a tool? No
AI: The artist with the second top song on Spotify in April 2023 was IU, Agust D.

> Finished chain.


As we can see, it answers question well from the SQL database. How does it do this? Let's look at the prompt.

In [34]:
print(conversational_agent.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

In [42]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

docstore = DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name = "Search",
        func = docstore.search,
        description = "search wikipedia"
    ),
    Tool(
        name = "Lookup",
        func = docstore.lookup,
        description = "lookup a term in wikipedia"
    )
]

In [43]:
docstore_agent = initialize_agent(
    tools,
    llm,
    agent = "react-docstore",
    verbose = True,
    max_iterations = 3
)

In [45]:
docstore_agent("Where is Karl Marx buried?")



> Entering new AgentExecutor chain...
Thought: I need to search Karl Marx and find where he is buried.
Action: Search[Karl Marx]
Observation: Could not find [Karl Marx]. Similar: ['Karl Marx', 'Tomb of Karl Marx', 'Karl-Marx-Allee', 'Karl Marx University', 'The Young Karl Marx', 'Karl-Marx-Hof', 'FC Erzgebirge Aue', 'Chemnitz', 'Marxism', 'Karl Marx House']
Thought:To find where Karl Marx is buried, I can search Tomb of Karl Marx.
Action: Search[Tomb of Karl Marx]
Observation: The Tomb of Karl Marx stands in the Eastern cemetery of Highgate Cemetery, North London, England. It commemorates the burial sites of Marx, of his wife, Jenny von Westphalen, and other members of his family. Originally buried in a different part of the Eastern cemetery, the bodies were disinterred and reburied at their present location in 1954. The tomb was designed by Laurence Bradshaw and was unveiled in 1956, in a ceremony led by Harry Pollitt, the General Secretary of the Communist Party of Great Britain, w

{'input': 'Where is Karl Marx buried?',
 'output': 'Eastern cemetery of Highgate Cemetery, North London, England'}

In [ ]:
import getpass

os.environ["SERP_API_KEY"] = getpass.getpass()

In [48]:
from langchain import SerpAPIWrapper

search = SerpAPIWrapper(serpapi_api_key = os.environ["SERP_API_KEY"]) #allows searching from google

In [49]:
tools = [
    Tool(
        name = "Intermediate Answer",
        func = search.run,
        description = 'google search'
    )
]

self_ask_with_search = initialize_agent(
    tools,
    llm,
    agent = "self-ask-with-search",
    verbose = True
)

In [50]:
self_ask_with_search(
    "What came first: Theory of mind, theory of the atom, or theory of the cell?"
)



> Entering new AgentExecutor chain...
Yes.
Follow up: When was the theory of mind first proposed?
Intermediate answer: 1978
Follow up: When was the theory of the atom first proposed?
Intermediate answer: 5th century bce
Follow up: When was the theory of the cell first proposed?
Intermediate answer: The classical cell theory was proposed by Theodor Schwann in 1839. There are three parts to this theory. The first part states that all organisms are made of cells.
So the final answer is: Theory of the atom

> Finished chain.


{'input': 'What came first: Theory of mind, theory of the atom, or theory of the cell?',
 'output': 'Theory of the atom'}

As we can see, our agent searches from Google and answers our question well.

We can combine the multiple tools we've given our agents above to get an agent that can accomplish tasks such as:
- Look at an SQL table
- Look up information on Wikipedia
- Make Google Searches